In [1]:
import pandas as pd
import numpy as np 
import datetime as dt
import re
import os
import seaborn as sns
import scipy.stats as stats

from sklearn.preprocessing import  RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor





In [2]:
raw = pd.read_csv(f'D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project//trip_data/yellow_tripdata_2016-06.csv')

In [4]:
raw.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,09/06/2016 21:06,09/06/2016 21:13,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,09/06/2016 21:06,09/06/2016 21:35,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,09/06/2016 21:06,09/06/2016 21:13,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,09/06/2016 21:06,09/06/2016 21:36,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,09/06/2016 21:06,09/06/2016 21:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


In [5]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1048575 non-null  int64  
 1   tpep_pickup_datetime   1048575 non-null  object 
 2   tpep_dropoff_datetime  1048575 non-null  object 
 3   passenger_count        1048575 non-null  int64  
 4   trip_distance          1048575 non-null  float64
 5   pickup_longitude       1048575 non-null  float64
 6   pickup_latitude        1048575 non-null  float64
 7   RatecodeID             1048575 non-null  int64  
 8   store_and_fwd_flag     1048575 non-null  object 
 9   dropoff_longitude      1048575 non-null  float64
 10  dropoff_latitude       1048575 non-null  float64
 11  payment_type           1048575 non-null  int64  
 12  fare_amount            1048575 non-null  float64
 13  extra                  1048575 non-null  float64
 14  mta_tax           

In [6]:
raw.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,1.528136e+00,1.636896e+00,3.010579e+00,-7.304573e+01,4.024021e+01,1.040175e+00,-7.311215e+01,4.027799e+01,1.333441e+00,1.365420e+01,3.558627e-01,4.973224e-01,1.895595e+00,3.316910e-01,2.996540e-01,1.703431e+01
std,4.992080e-01,1.301522e+00,3.750994e+00,8.232540e+00,4.535013e+00,4.787309e-01,7.934125e+00,4.369716e+00,4.888149e-01,1.304250e+02,4.574452e-01,3.936377e-02,2.656670e+00,1.798481e+00,1.358867e-02,1.306874e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,-1.151684e+02,0.000000e+00,1.000000e+00,-1.151754e+02,0.000000e+00,1.000000e+00,-1.750000e+02,-4.500000e+00,-5.000000e-01,-2.600000e+01,-1.050000e+01,-3.000000e-01,-1.753000e+02
25%,1.000000e+00,1.000000e+00,1.000000e+00,-7.399156e+01,4.073714e+01,1.000000e+00,-7.399071e+01,4.073568e+01,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.800000e+00
50%,2.000000e+00,1.000000e+00,1.700000e+00,-7.398096e+01,4.075434e+01,1.000000e+00,-7.397849e+01,4.075486e+01,1.000000e+00,1.000000e+01,5.000000e-01,5.000000e-01,1.450000e+00,0.000000e+00,3.000000e-01,1.236000e+01
75%,2.000000e+00,2.000000e+00,3.230000e+00,-7.396574e+01,4.076863e+01,1.000000e+00,-7.396120e+01,4.077002e+01,2.000000e+00,1.550000e+01,5.000000e-01,5.000000e-01,2.550000e+00,0.000000e+00,3.000000e-01,1.896000e+01
max,2.000000e+00,9.000000e+00,5.000000e+02,0.000000e+00,5.109830e+01,9.900000e+01,1.062469e+02,5.031214e+01,4.000000e+00,1.330578e+05,7.000000e+01,5.000000e-01,3.957000e+02,5.855800e+02,5.200000e-01,1.330637e+05


In [7]:
# TRANSFORM OBJECT DTYPE TO DATETIME
raw['tpep_pickup_datetime']=pd.to_datetime(raw['tpep_pickup_datetime'])
raw['tpep_dropoff_datetime']=pd.to_datetime(raw['tpep_dropoff_datetime'])

In [8]:
raw['duration']=(raw['tpep_dropoff_datetime']-raw['tpep_pickup_datetime']).dt.total_seconds()

### HANDLING UNREALISTIC DATA

In [9]:
# Location conditions 
# longitude
idxlo=raw['pickup_longitude']<-73 
idx1lo =raw['pickup_longitude']>-75
idxlo1=raw['dropoff_longitude']<-73 
idx1lo1=raw['dropoff_longitude']>-75

#Latitude
idxla = raw['pickup_latitude']>40.3
idx1la = raw['pickup_latitude']<41
idxla1 = raw['dropoff_latitude']>40.3
idx1la1 = raw['dropoff_latitude']<41



In [10]:
rawlolafiltered= raw[idxlo & idx1lo & idxlo1 &idx1lo1 & idxla & idx1la &idxla1 & idx1la1]

In [11]:
rawlolafiltered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.00,0.3,7.30,420.0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.00,0.3,27.30,1740.0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.00,0.3,9.36,420.0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.00,0.3,28.30,1800.0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.00,0.3,17.76,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,2016-03-06 11:15:00,2016-03-06 11:22:00,1,0.42,-73.979027,40.745171,1,N,-73.985802,40.746269,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16,420.0
1048571,2,2016-03-06 11:15:00,2016-03-06 11:46:00,1,5.80,-73.951935,40.790062,1,N,-73.994850,40.752895,1,25.0,0.0,0.5,5.16,0.00,0.3,30.96,1860.0
1048572,1,2016-03-06 11:06:00,2016-03-06 11:32:00,2,6.70,-73.995590,40.687832,1,N,-74.006721,40.751347,1,23.5,0.0,0.5,5.95,5.54,0.3,35.79,1560.0
1048573,2,2016-03-06 11:06:00,2016-03-06 11:24:00,1,1.80,-73.983688,40.734360,1,N,-74.002777,40.749432,1,12.0,0.0,0.5,2.56,0.00,0.3,15.36,1080.0


In [15]:
#DEALING WITH THE FEATURES WHOSE VALUES ARE ONLY MEANINGFUL WHEN >0
c01=  rawlolafiltered['duration']>0
c02= rawlolafiltered['passenger_count']>0
c03=rawlolafiltered['extra']>=0
c04=rawlolafiltered['mta_tax']>=0
c05=rawlolafiltered['tip_amount']>=0
c06=rawlolafiltered['tolls_amount']>=0
c07=rawlolafiltered['improvement_surcharge']>=0
c08=rawlolafiltered['total_amount']>=0
c09=rawlolafiltered['improvement_surcharge']>=0
rawlolafiltered= rawlolafiltered[ c01 & c02 & c03 & c04 & c05 & c06 & c07 & c08 & c09]


In [16]:
rawlolafiltered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.00,0.3,7.30,420.0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.00,0.3,27.30,1740.0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.00,0.3,9.36,420.0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.00,0.3,28.30,1800.0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.00,0.3,17.76,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,2016-03-06 11:15:00,2016-03-06 11:22:00,1,0.42,-73.979027,40.745171,1,N,-73.985802,40.746269,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16,420.0
1048571,2,2016-03-06 11:15:00,2016-03-06 11:46:00,1,5.80,-73.951935,40.790062,1,N,-73.994850,40.752895,1,25.0,0.0,0.5,5.16,0.00,0.3,30.96,1860.0
1048572,1,2016-03-06 11:06:00,2016-03-06 11:32:00,2,6.70,-73.995590,40.687832,1,N,-74.006721,40.751347,1,23.5,0.0,0.5,5.95,5.54,0.3,35.79,1560.0
1048573,2,2016-03-06 11:06:00,2016-03-06 11:24:00,1,1.80,-73.983688,40.734360,1,N,-74.002777,40.749432,1,12.0,0.0,0.5,2.56,0.00,0.3,15.36,1080.0


In [17]:
# Speed conditions
sc1= (rawlolafiltered['trip_distance']/rawlolafiltered['duration'])<=0.009 # < 38MPH
sc2= (rawlolafiltered['trip_distance']/rawlolafiltered['duration'])>0.00097 # >3.5MPH
rawlolafiltered1= rawlolafiltered[sc1 & sc2]

In [18]:
#Fare amount conditions
fc1= rawlolafiltered['fare_amount']<rawlolafiltered['trip_distance']*10
fc2= rawlolafiltered['fare_amount']>rawlolafiltered['trip_distance']*2.5
rawlolafiltered2= rawlolafiltered1[fc1 & fc2]

<ipython-input-18-7bc0319042de>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rawlolafiltered2= rawlolafiltered1[fc1 & fc2]


In [19]:
raw= rawlolafiltered2

In [68]:
raw

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,seasons,start_dayparts,sdawn,sevening,smidnight,smorning,snoon,Fall,Spring,Winter
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,...,Fall,sevening,0,1,0,0,0,1,0,0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,...,Fall,sevening,0,1,0,0,0,1,0,0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,...,Fall,sevening,0,1,0,0,0,1,0,0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,...,Fall,sevening,0,1,0,0,0,1,0,0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,...,Fall,sevening,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,2,2016-03-06 11:15:00,2016-03-06 11:27:00,1,1.42,-73.958763,40.778252,1,N,-73.978683,...,Spring,snoon,0,0,0,0,1,0,1,0
1048571,2,2016-03-06 11:15:00,2016-03-06 11:46:00,1,5.80,-73.951935,40.790062,1,N,-73.994850,...,Spring,snoon,0,0,0,0,1,0,1,0
1048572,1,2016-03-06 11:06:00,2016-03-06 11:32:00,2,6.70,-73.995590,40.687832,1,N,-74.006721,...,Spring,snoon,0,0,0,0,1,0,1,0
1048573,2,2016-03-06 11:06:00,2016-03-06 11:24:00,1,1.80,-73.983688,40.734360,1,N,-74.002777,...,Spring,snoon,0,0,0,0,1,0,1,0


# AUGMENTED DATASET

In [21]:

raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
months =['January','February','March', 'April','May','June','July','August','September','October','November','December']
dmonths= dict(zip(months,np.arange(1,13,1)))
raw['start_month_num']=raw['start_month'].map(dmonths)
raw['end_month_num']=raw['end_month'].map(dmonths)

raw['start_day']=raw['tpep_pickup_datetime'].dt.dayofweek
raw['end_day']=raw['tpep_dropoff_datetime'].dt.dayofweek
dday= dict(zip(np.arange(0,7,1),[0,0,0,0,0,1,1]))
raw['weekend_starttrip']=raw['start_day'].map(dday)
raw['weekend_endtrip']=raw['end_day'].map(dday)

raw['start_hour']=raw['tpep_pickup_datetime'].dt.hour
raw['end_hour']=raw['tpep_dropoff_datetime'].dt.hour







<ipython-input-21-2c4379d363cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
<ipython-input-21-2c4379d363cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
<ipython-input-21-2c4379d363cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

### SIN COS TRANSFORM FOR PERIODICITY FEATURES

In [22]:

raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox

raw['start_day_sin']= np.sin(raw['start_day']*(2.*np.pi/7))
raw['start_day_cos']= np.cos(raw['start_day']*(2.*np.pi/7))


raw['start_month_sin']= np.sin(raw['start_month_num']*(2.*np.pi/12))
raw['start_month_cos']= np.cos(raw['start_month_num']*(2.*np.pi/12))


<ipython-input-22-e426ed6db2a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
<ipython-input-22-e426ed6db2a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox
<ipython-input-22-e426ed6db2a0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [23]:
raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,weekend_starttrip,weekend_endtrip,start_hour,end_hour,start_hour_sin,start_hour_cos,start_day_sin,start_day_cos,start_month_sin,start_month_cos
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16


In [24]:
# BUILD SEASONS AND DAYPARTS FEATURES BASED ON TIME 
def seasons(month):
    if month in [11,12,1]:
        return 'Winter'
    elif month in [2,3,4]:
        return 'Spring'
    elif month in [5,6,7]:
        return 'Summer'
    else: 
        return 'Fall'


In [25]:
raw['seasons']=raw['start_month_num'].map(seasons)

<ipython-input-25-61ef6c99acb5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['seasons']=raw['start_month_num'].map(seasons)


In [26]:
rawseasons=pd.get_dummies(raw['seasons'])

In [27]:
rawseasons

,Fall,Spring,Winter
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
1048569,0,1,0
1048571,0,1,0
1048572,0,1,0
1048573,0,1,0


In [28]:
hours= np.arange(0,24)
sdayparts=['smidnight','smidnight','sdawn','sdawn','sdawn','sdawn','smorning','smorning','smorning','smorning','snoon','snoon','snoon','snoon','safternoon','safternoon','safternoon','safternoon','sevening','sevening','sevening','sevening','smidnight','smidnight']
dsdayparts= dict(zip(hours,sdayparts))


In [29]:
raw['start_dayparts']=raw['start_hour'].map(dsdayparts)

<ipython-input-29-2c3e7cdc998a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_dayparts']=raw['start_hour'].map(dsdayparts)


In [30]:
start_dayparts = pd.get_dummies(raw['start_dayparts'],drop_first=True)

In [31]:
start_dayparts

,sdawn,sevening,smidnight,smorning,snoon
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
1048569,0,0,0,0,1
1048571,0,0,0,0,1
1048572,0,0,0,0,1
1048573,0,0,0,0,1


In [33]:
raw= pd.concat((raw,start_dayparts, rawseasons),axis=1)

In [34]:
raw=raw.loc[:,~raw.columns.duplicated()]

In [35]:
raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,seasons,start_dayparts,sdawn,sevening,smidnight,smorning,snoon,Fall,Spring,Winter
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,...,Fall,sevening,0,1,0,0,0,1,0,0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,...,Fall,sevening,0,1,0,0,0,1,0,0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,...,Fall,sevening,0,1,0,0,0,1,0,0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,...,Fall,sevening,0,1,0,0,0,1,0,0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,...,Fall,sevening,0,1,0,0,0,1,0,0


### LONGTITUDE LATITUDE FEATURES HANDLING

In [36]:
from math import radians

In [37]:
###LON LAT OF SOME SPECIFIC PLACES
JFKairport= (-73.780968, 40.641766)

NWairport=(-74.172363, 40.735657)

#MIDTOWN MANHATTAN
Mark= (-73.96342029243233,40.77541739176158)
Timesquare = (-73.98604331729206,40.75837054036599)
Grand =( -73.97711770261203,40.75284316174533)
Penn= (-73.99368229963214, 40.75069857282476)
Lincoln= (-73.9839037407981, 40.773072825611635)

#WEST SIDE MANHATTAN Bloomingdale flor de mayo 40.797800366747595, -73.9700034852982
Flor = (-73.9700034852982,40.797800366747595)

# LOWER MANHATTAN one world trade center 40.71315120718066, -74.01335094122703
Wtc=  (-74.01335094122703,40.71315120718066)

#UPPER MANHATTAN bank of america 40.82320010657351, -73.9420433120473
Boa = (-73.9420433120473, 40.82320010657351)

L= [JFKairport, NWairport, Mark,Timesquare,Grand,Penn,Lincoln,Flor,Wtc,Boa]

In [38]:
#Converting decimal lon lat to radians for each specific places mentioned above
def detora(List):
    res=[]
    for i in List:
        listc= list(map(radians,i))
        res.append(listc)
    return res
res= detora(L)

In [39]:
res

[[-1.2877208169085756, 0.7093326305250857],
 [-1.294551948334419, 0.7109713376130758],
 [-1.2909052101393315, 0.7116652873611982],
 [-1.2913000564098942, 0.7113677636772469],
 [-1.2911442750459634, 0.7112712927213121],
 [-1.2914333818032302, 0.7112338625835463],
 [-1.2912627137554762, 0.7116243669623508],
 [-1.2910201085301345, 0.7120559439710956],
 [-1.2917766643584558, 0.7105785374276066],
 [-1.2905321114474784, 0.7124992530602078]]

In [40]:
# convert decimal lon lat to radians for PUlon and PUlat in the 'raw'  dataframe
puloc= pd.Series(map(radians,raw['pickup_longitude']))
pulac= pd.Series(map(radians,raw['pickup_latitude']))

In [41]:
puloc

0        -1.291253
1        -1.291225
2        -1.291444
3        -1.291236
4        -1.291319
            ...   
918745   -1.290824
918746   -1.290705
918747   -1.291467
918748   -1.291259
918749   -1.291053
Length: 918750, dtype: float64

In [43]:
#Compute the Harvesine distance between the datapoints and each specific place listed above
def haversineindf(df):
    for i in np.arange(0,10):
        dlon= puloc-res[i][0]
        dlat= pulac-res[i][1]
        a= np.sin(dlat/2)**2 + np.cos(pulac) * np.cos(res[0][1]) * np.sin(dlon/2)**2
        c= 2*np.arcsin(np.sqrt(a))
        s= pd.Series(c*3956).rename(i)
        df= pd.concat((df,s),axis=1)
    return df

In [45]:
s1= pd.Series(np.zeros(918750))
df= haversineindf(s1)

In [46]:
df=df.iloc[:,1::]

In [66]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,13.414098,10.046033,1.445355,0.226093,0.647347,0.889734,0.838421,2.639536,3.653793,4.812342
1,12.385433,9.981167,2.841849,1.515480,1.142539,1.153575,2.516204,4.265262,2.319214,6.325456
2,13.479301,9.381028,2.332671,0.664452,0.908575,0.042016,1.613888,3.468332,2.801447,5.682368
3,13.940239,10.289019,0.996919,1.088839,1.478966,1.707475,0.098570,1.772977,4.496559,4.005461
4,12.502349,9.700696,3.169423,1.740634,1.455313,1.258058,2.759965,4.551141,1.949136,6.648599


In [67]:
df.describe()

,0,1,2,3,4,5,6,7,8,9
count,918750.000000,918750.000000,918750.000000,918750.000000,918750.000000,918750.000000,918750.000000,918750.000000,918750.000000,918750.000000
mean,12.777919,10.590286,2.472052,1.955244,1.900193,2.037097,2.334027,3.564727,3.696240,5.468047
std,1.894452,2.032313,2.018047,1.996020,1.901876,2.028453,2.048835,2.160422,2.205098,2.113637
min,0.071535,1.711364,0.002537,0.001460,0.002053,0.004188,0.025786,0.007640,0.002860,0.009395
25%,12.642307,9.505204,1.110010,0.805360,0.806188,0.907061,1.210884,2.227867,2.201114,4.187285
50%,13.028093,10.166448,2.012250,1.492853,1.476428,1.503005,1.733359,3.212865,3.427542,5.249694
75%,13.457517,11.089749,3.292234,2.342020,2.263726,2.517691,2.851632,4.550002,4.782631,6.545152
max,62.196045,52.130094,56.320407,54.699775,54.915072,54.087000,55.314515,56.831945,51.925404,59.012929


In [48]:
scale= RobustScaler() # due to outliers in  df
scale.fit(df)


RobustScaler()

In [49]:
dfscaled= scale.transform(df)

In [50]:
dfscaled= pd.DataFrame(dfscaled.tolist(),index=raw.index)

In [51]:
dfscaled

,0,1,2,3,4,5,6,7,8,9
0,0.473504,-0.075994,-0.259779,-0.824359,-0.568823,-0.380764,-0.545445,-0.246897,0.087643,-0.185486
1,-0.788337,-0.116931,0.380162,0.014725,-0.229078,-0.216952,0.477127,0.453203,-0.429332,0.456244
2,0.553486,-0.495675,0.146832,-0.539092,-0.389597,-0.907091,-0.072815,0.110014,-0.242530,0.183502
3,1.118910,0.077354,-0.465273,-0.262917,0.001742,0.126950,-0.996368,-0.620071,0.414104,-0.527694
4,-0.644919,-0.293934,0.530273,0.161246,-0.014487,-0.152081,0.625694,0.576313,-0.572689,0.593293
...,...,...,...,...,...,...,...,...,...,...
1048569,0.264492,0.879262,-0.778870,0.317419,0.359305,0.704712,-0.225408,-0.749519,0.735406,-0.858914
1048571,0.698488,1.240363,-0.383072,0.866168,0.968472,1.231882,0.188891,-0.915866,1.076770,-1.231552
1048572,-1.651486,-0.212704,1.954625,2.214610,2.137418,1.762534,2.549971,1.936701,-0.560638,1.912111
1048573,-0.782502,-0.181756,0.464974,0.110341,-0.106127,-0.161087,0.572677,0.527797,-0.500887,0.534501


In [52]:
dfscaled=dfscaled.rename(columns=dict(zip(np.arange(0,10),['tojfk','tonw', 'tomark','totimes','togrand','topenn','tolincoln','toflor','towtc','toboa'])))

In [53]:
#the haversine distance computed represented by a dataframe with informative column names
dfscaled

,tojfk,tonw,tomark,totimes,togrand,topenn,tolincoln,toflor,towtc,toboa
0,0.473504,-0.075994,-0.259779,-0.824359,-0.568823,-0.380764,-0.545445,-0.246897,0.087643,-0.185486
1,-0.788337,-0.116931,0.380162,0.014725,-0.229078,-0.216952,0.477127,0.453203,-0.429332,0.456244
2,0.553486,-0.495675,0.146832,-0.539092,-0.389597,-0.907091,-0.072815,0.110014,-0.242530,0.183502
3,1.118910,0.077354,-0.465273,-0.262917,0.001742,0.126950,-0.996368,-0.620071,0.414104,-0.527694
4,-0.644919,-0.293934,0.530273,0.161246,-0.014487,-0.152081,0.625694,0.576313,-0.572689,0.593293
...,...,...,...,...,...,...,...,...,...,...
1048569,0.264492,0.879262,-0.778870,0.317419,0.359305,0.704712,-0.225408,-0.749519,0.735406,-0.858914
1048571,0.698488,1.240363,-0.383072,0.866168,0.968472,1.231882,0.188891,-0.915866,1.076770,-1.231552
1048572,-1.651486,-0.212704,1.954625,2.214610,2.137418,1.762534,2.549971,1.936701,-0.560638,1.912111
1048573,-0.782502,-0.181756,0.464974,0.110341,-0.106127,-0.161087,0.572677,0.527797,-0.500887,0.534501


In [54]:
X= pd.concat((raw,dfscaled),axis=1)


### PREDICTING TRIP DISTANCE

In [55]:
ySGD= X['trip_distance']

In [56]:
X.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'duration', 'start_month',
       'end_month', 'start_month_num', 'end_month_num', 'start_day', 'end_day',
       'weekend_starttrip', 'weekend_endtrip', 'start_hour', 'end_hour',
       'start_hour_sin', 'start_hour_cos', 'start_day_sin', 'start_day_cos',
       'start_month_sin', 'start_month_cos', 'seasons', 'start_dayparts',
       'sdawn', 'sevening', 'smidnight', 'smorning', 'snoon', 'Fall', 'Spring',
       'Winter', 'tojfk', 'tonw', 'tomark', 'totimes', 'togrand', 'topenn',
       'tolincoln', 'toflor', 'towtc', 'toboa'],
      dtype='object')

In [57]:
#Select only columns of interest
XSGD=X[['start_hour_sin',        'start_hour_cos',
               'start_day_sin',         'start_day_cos',
             'start_month_sin',       'start_month_cos',
                       'sdawn',              'sevening',
                   'smidnight',              'smorning',
                       'snoon',                  'Fall',
                      'Spring',                'Winter',
            'tojfk','tonw', 'tomark','totimes','togrand','topenn','tolincoln','toflor','towtc','toboa']]


In [58]:
XSGD

,start_hour_sin,start_hour_cos,start_day_sin,start_day_cos,start_month_sin,start_month_cos,sdawn,sevening,smidnight,smorning,...,tojfk,tonw,tomark,totimes,togrand,topenn,tolincoln,toflor,towtc,toboa
0,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,0.473504,-0.075994,-0.259779,-0.824359,-0.568823,-0.380764,-0.545445,-0.246897,0.087643,-0.185486
1,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,-0.788337,-0.116931,0.380162,0.014725,-0.229078,-0.216952,0.477127,0.453203,-0.429332,0.456244
2,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,0.553486,-0.495675,0.146832,-0.539092,-0.389597,-0.907091,-0.072815,0.110014,-0.242530,0.183502
3,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,1.118910,0.077354,-0.465273,-0.262917,0.001742,0.126950,-0.996368,-0.620071,0.414104,-0.527694
4,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,-0.644919,-0.293934,0.530273,0.161246,-0.014487,-0.152081,0.625694,0.576313,-0.572689,0.593293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,0.264492,0.879262,-0.778870,0.317419,0.359305,0.704712,-0.225408,-0.749519,0.735406,-0.858914
1048571,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,0.698488,1.240363,-0.383072,0.866168,0.968472,1.231882,0.188891,-0.915866,1.076770,-1.231552
1048572,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,-1.651486,-0.212704,1.954625,2.214610,2.137418,1.762534,2.549971,1.936701,-0.560638,1.912111
1048573,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,-0.782502,-0.181756,0.464974,0.110341,-0.106127,-0.161087,0.572677,0.527797,-0.500887,0.534501


In [59]:
Xtrain,Xtest,ytrain,ytest=train_test_split(XSGD,ySGD,test_size=0.2)

In [60]:
model = SGDRegressor()

In [61]:
model.fit(Xtrain,ytrain)

SGDRegressor()

In [62]:
print(model.score(Xtrain,ytrain))

0.35753653516280537


In [63]:
print(model.score(Xtest, ytest))

0.353047393229145


In [65]:
X.corr()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,...,tojfk,tonw,tomark,totimes,togrand,topenn,tolincoln,toflor,towtc,toboa
VendorID,1.000000,0.302714,0.006018,0.018056,0.004160,0.003938,0.003013,0.003346,-0.005182,0.001211,...,-0.010474,0.018949,0.003354,0.013099,0.011998,0.016526,0.008864,0.001881,0.018363,-0.002968
passenger_count,0.302714,1.000000,0.014592,0.012406,-0.002908,0.004360,0.004201,-0.000762,0.016407,0.014150,...,-0.010770,0.012409,0.009931,0.012704,0.012889,0.013176,0.011486,0.008140,0.010267,0.005095
trip_distance,0.006018,0.014592,1.000000,0.506305,-0.281175,0.384432,0.357831,-0.184948,-0.035280,0.955960,...,-0.541926,0.498058,0.488596,0.539868,0.534490,0.529853,0.532422,0.454092,0.374934,0.354192
pickup_longitude,0.018056,0.012406,0.506305,1.000000,-0.056739,0.232869,0.262597,0.051960,0.047665,0.450465,...,-0.793081,0.988197,0.417112,0.757174,0.737254,0.853396,0.616724,0.358046,0.887337,0.143334
pickup_latitude,0.004160,-0.002908,-0.281175,-0.056739,1.000000,-0.173475,0.064792,0.433892,0.016612,-0.241948,...,0.634588,-0.022036,-0.814434,-0.455968,-0.402735,-0.260960,-0.698471,-0.912269,0.300914,-0.978966
RatecodeID,0.003938,0.004360,0.384432,0.232869,-0.173475,1.000000,0.068627,-0.084935,0.003140,0.390863,...,-0.291425,0.232640,0.233367,0.244576,0.244211,0.239979,0.245255,0.224339,0.183302,0.198909
dropoff_longitude,0.003013,0.004201,0.357831,0.262597,0.064792,0.068627,1.000000,0.174673,0.048385,0.299816,...,-0.160421,0.261692,0.030938,0.159813,0.157801,0.208849,0.097976,0.007828,0.261919,-0.053081
dropoff_latitude,0.003346,-0.000762,-0.184948,0.051960,0.433892,-0.084935,0.174673,1.000000,0.036227,-0.182996,...,0.208257,0.075067,-0.318961,-0.120070,-0.089694,-0.022053,-0.253963,-0.378279,0.224135,-0.416099
payment_type,-0.005182,0.016407,-0.035280,0.047665,0.016612,0.003140,0.048385,0.036227,1.000000,-0.039905,...,-0.028927,0.051870,0.004048,0.025839,0.031409,0.037857,0.013800,-0.003390,0.061635,-0.014181
fare_amount,0.001211,0.014150,0.955960,0.450465,-0.241948,0.390863,0.299816,-0.182996,-0.039905,1.000000,...,-0.474984,0.441550,0.431121,0.476657,0.472668,0.468477,0.469698,0.401133,0.331646,0.310113
